# Assessment Problems

https://github.com/ranaroussi/yfinance

## Imports

In [24]:
# Date and time handling
import datetime as dt

# Data manipulation
import pandas as pd

# Yahoo Finance data
import yfinance as yf

## Problem 1: Data from yfinance

In [32]:
# Fetch stocks tickers for analysis
tickers = yf.Tickers('META AAPL AMZN NFLX GOOG')
tickers

yfinance.Tickers object <META,AAPL,AMZN,NFLX,GOOG>

### Configure filename

In order to create descriptive filename, we need to get today's date. We will use `now()` classmethod within [datetime](https://docs.python.org/3/library/datetime.html#datetime.datetime) module. `now()` allows to get current local date and time. 

In [26]:
# Get today's date 
now = dt.datetime.now()

# Display today's date
now

datetime.datetime(2025, 11, 17, 8, 54, 17, 945180)

In order to format the date and time for file names, refer to `strftime()` [Format Codes](https://docs.python.org/3/library/datetime.html#format-codes) python official documentation.

In [27]:
# Format date and time
now.strftime("%Y%m%d-%H%M%S")

'20251117-085417'

Setting the filename:
1. Specify the folder path. Since the data folder and this Jupyter notebook are in the same directory, we can refer to the folder directly by its name. Therefore, the path to the folder is "data/"
2. Generate a timestamp for the current date and time using the `datetime.now()` method, and format it with `strftime()` to produce the YYYYMMDD-HHmmss pattern.
3. Add the ".csv" extension to the filename indicate the file format and make it easily recognisable as a CSV file.

In [33]:
# File name
fileneme = 'data/' + dt.datetime.now().strftime('%Y%m%d-%H%M%S') + '.csv'

fileneme

'data/20251117-090311.csv'

In [ ]:
# Download historical data for the last 5 days
data = yf.download(('META AAPL AMZN NFLX GOOG'), period='5D', interval='1h')

# Display the first 3 rows of the data
print(data.head(3))

/tmp/ipykernel_8677/2840011167.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(('META AAPL AMZN NFLX GOOG'), period='5D', interval='1h')
[*********************100%***********************]  5 of 5 completed

Price                           Close                                      \
Ticker                           AAPL        AMZN        GOOG        META   
Datetime                                                                    
2025-11-10 14:30:00+00:00  271.856995  248.029999  288.720001  633.837402   
2025-11-10 15:30:00+00:00  269.066406  246.380005  286.965515  630.409973   
2025-11-10 16:30:00+00:00  269.079987  247.104996  287.309998  633.864990   

Price                                         High                          \
Ticker                            NFLX        AAPL        AMZN        GOOG   
Datetime                                                                     
2025-11-10 14:30:00+00:00  1124.795044  273.730011  251.750000  290.429993   
2025-11-10 15:30:00+00:00  1116.630005  271.989990  248.089996  289.269897   
2025-11-10 16:30:00+00:00  1116.839844  269.758789  247.445007  287.579987   

Price                                               ...        Open 

Now we can combine the above steps within a single function that downloads all hourly data for the previous five days for the five FAANG stocks.

In [36]:
# Function to get data and save to CSV
def get_data():
    # Get historical data for the last 5 days
    data = yf.download(('META AAPL AMZN NFLX GOOG'), period='5D', interval='1h')
    
    # Create a filename with the current date and time
    filename = 'data/' + dt.datetime.now().strftime('%Y%m%d-%H%M%S') + '.csv'
    
    # Save data to CSV
    data.to_csv(filename)
    
    return data

# Call the function to get data and save to CSV
faang_data = get_data()

/tmp/ipykernel_8677/3740010575.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(('META AAPL AMZN NFLX GOOG'), period='5D', interval='1h')
[*********************100%***********************]  5 of 5 completed
